In [155]:
import pandas as pd

In [156]:
import numpy as np

In [157]:
data = pd.read_csv("properati_data_fixed_all_years_duplicated_with_outliers.csv", compression = "gzip")

In [158]:
#Los que tienen nulo (room, surface total), (room, cubierta), (cubierta, total) no nos sirven demasiado
len(data)

1210782

In [159]:
data.dropna(axis = 0,how='all', subset=["rooms","surface_total_in_m2"], inplace = True)
data.dropna(axis = 0,how='all', subset=["rooms","surface_covered_in_m2"], inplace = True)
data.dropna(axis = 0,how='all', subset=["surface_covered_in_m2","surface_total_in_m2"], inplace = True)

In [160]:
len(data)

584301

In [161]:
data["year"].value_counts()

2017.0    229943
2016.0     87654
2014.0     83136
2015.0     81912
2013.0     75804
2012.0      7714
Name: year, dtype: int64

In [162]:
data_2017 = data[data["year"] == 2017]

In [163]:
len(data_2017)

229943

In [164]:
data_2017["state_name"].value_counts()

Bs.As. G.B.A. Zona Norte    86017
Capital Federal             85565
Bs.As. G.B.A. Zona Sur      31822
Bs.As. G.B.A. Zona Oeste    26539
Name: state_name, dtype: int64

In [165]:
#Sacamos las de Zona Norte porque no tenemos que predecir casi ninguna de esa zona.
data_2017_zo_zs_caba = data_2017[(data_2017["state_name"] == "Capital Federal") | (data_2017["state_name"] == "Bs.As. G.B.A. Zona Sur") | (data_2017["state_name"] == "Bs.As. G.B.A. Zona Oeste")]

In [166]:
data_2017_zo_zs_caba["state_name"].value_counts()

Capital Federal             85565
Bs.As. G.B.A. Zona Sur      31822
Bs.As. G.B.A. Zona Oeste    26539
Name: state_name, dtype: int64

In [167]:
data_2017_zo_zs_caba.isnull().sum()

created_on                      0
description                     0
expenses                   119745
floor                      119950
id                              0
lat-lon                     37493
place_name                   1897
place_with_parent_names         0
price_aprox_usd                 0
property_type                   0
rooms                       51357
state_name                      0
surface_covered_in_m2        3660
surface_total_in_m2            72
year                            0
dtype: int64

In [168]:
#A los de rooms nulas tratamos de rellenarlas con con los que tengan superficie total y cubierta parecida (su promedio)

In [169]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"].isnull()]["surface_covered_in_m2"].values[4]

124.0

In [170]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 9]["description"].values[3]

'codigo: 3016-wp0141 ubicado en: malabia 1221 9no piso -  publicado por: waiman propiedades. el precio es de usd 340000 null. hermoso departamento reciclado a nuevo con gran balc\xc3\x93n terraza con parrilla. dormitorio principal en suite con vestidor . publicado a trav\xc3\xa9s de mapaprop'

In [171]:
data_2017_zo_zs_caba["rooms"].value_counts()

2.0       25898
3.0       24032
1.0       21470
4.0       14213
5.0        4205
6.0        1342
7.0         613
8.0         292
10.0        143
9.0         127
11.0         38
13.0         30
14.0         26
12.0         26
22.0         19
17.0         13
16.0         10
103.0         5
2.5           5
106.0         4
104.0         4
18.0          4
102.0         4
24.0          3
19.0          3
26.0          3
31.0          3
57.0          3
29.0          3
105.0         3
210.0         2
242.0         2
504.0         2
47.0          2
82.0          2
3.5           2
21.0          2
1613.0        1
551.0         1
801.0         1
303.0         1
236.0         1
15.0          1
64.0          1
40.0          1
136.0         1
43.0          1
59.0          1
Name: rooms, dtype: int64

In [172]:
#Con esto vemos que ya mas de 8 ambientes es algo raro, 

In [173]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 10]["description"].values[3]

'corredor responsable: uno bienes raices srl - cucicba 4993contacto: viviana casamiquela - mls id # 420491037-44\xc2\xa1ideal inversores! \xc2\xa1excelente valor por metro cuadrado en la zona! - u$s 2,333  (actualizado 15-6-2017) ya est\xc3\x81n terminados - apto profesional  departamento de excelente calidad en planta muy bien resuelta. cocina separada. estar comedor con balc\xc3\xb3n. espacios amplios y luminosos.  espacio para servicio de seguridad. amenities: parrillas y solarium. todos estos espacios comunes ubicados en 10\xc2\xb0 piso. servicios: laundry.   \xc2\xa1no es apto cr\xc3\x89dito!  posibilidad de compra de cocheras en edificio a partir de u$s 25.000 (ideal inversionistas)'

In [174]:
posibles_descripciones_ambientes = {"1 ambiente": 1, "monoambiente": 1, "mono ambiente":1, "2 ambientes": 2,"3 ambientes": 3,
                          "4 ambientes": 4,"5 ambientes": 5, "6 ambientes": 6, "7 ambientes": 7,"8 ambientes": 8,
                          "9 ambientes": 1, "10 ambientes": 10,
                         "ambientes: 1": 1, "ambientes: 2": 2, "ambientes: 3": 3, "ambientes: 4": 4, "ambientes: 5": 5,
                         "ambientes: 6": 6, "ambientes: 7": 7, "un ambiente": 1, "dos ambientes": 2, "tres ambientes": 3,
                         "cuatro ambientes": 4,"cinco ambientes": 5," seis ambientes": 6, "siete ambientes": 7,
                         "2 impecables ambientes": 2, "3 impecables ambientes": 3, "4 amplios ambientes": 4, "2 amplios ambientes": 2,
                          "3 amplios ambientes": 3, "5 amplios ambientes": 5, "2  ambientes": 2, "3  ambientes": 3,
                          "2 grandes ambientes": 2, "6 amplios ambientes": 6, "1 ambientes": 1
                         ,"3 amplios  ambientes": 3, "3 y ½ ambientes": 3.5,"2 y ½ ambientes": 2.5, "2 y medio ambiente": 2.5,
                         "(2) ambientes":2, "2 amplisimos ambientes": 2,"4  ambientes":4,"3 grandes ambientes":3,
                                   "4 ½ ambientes": 4.5, "4 cÃ²modos y confortables ambientes":4, "cantidad de ambientes 4":4,
                                   "cantidad de ambientes 3": 3,"ambientes : 4":4,"ambientes : 5":5,"ambientes : 3":3,"ambientes : 2":2,
                                   "ambientes : 1":1,"chalet de 5  ambientes":5,"4 cÃ³modos y confortables ambientes":4,"4   ambientes":4,
                                   "3 buenos ambientes":3,"4 còmodos y confortables ambientes":4,"2 amb.":2,"3 amb.":3,"4 amb.":4,"1 amb.":1, "5 amb.":5,"6 amb.":6,"7 amb.":7,
                                   "3 (tres) ambientes": 3, "2 (dos) ambientes":2, "4 (cuatro) ambientes":4, "5 (cinco) ambientes": 5, "1 (uno) ambientes": 1, "1 (uno) ambiente":1,
                                   "6 (seis) ambientes":6, "7 (siete) ambientes":7, "8 (ocho) ambientes":8, "9 (nueve) ambientes":9, "10 (diez) ambientes":10, "1 amb":1, "2 amb":2,
                                   "3 amb":3, "4 amb":4, "5 amb":5, "6 amb":6,"7 amb":7,"8 amb":8}


In [175]:

def fix_ambientes(row, search_title, search_extra, search_description):
    
    if pd.isnull(row["rooms"]):
        return np.nan#Ya fueron fixeadas
    cols = []
    if search_description:
        descripcion = row["description"]
        cols.append(descripcion)
    if search_extra:
        extra = row["extra"]
        cols.append(extra)
    if search_title:
        title = row["title"]
        cols.append(title)
        
    ambiente_actual = row["rooms"]
    for col in cols:
        if not(pd.isnull(col)):
            for k in posibles_descripciones_ambientes:
        
                if k in col:
                    value_found = float(posibles_descripciones_ambientes[k])
                    if float(row["rooms"]) != value_found:
                        return value_found
    return row["rooms"]

In [176]:
#Antes vuelvo a aplicar esto xq habia algunos que se pueden corregir
data_2017_zo_zs_caba["rooms"] = data_2017_zo_zs_caba.apply(lambda row: fix_ambientes(row, False, False, True), axis = 1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [177]:
#Pongo en NaN aquellos que tengan mas de 8 ambientes

In [178]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == 9]["description"].values

array([ '- edificio inteligente- pisos exclusivos, - hall privado - placar de recepci\xc3\xb3n.- 2 dormitorios (1 en suite) + escritorio.- 2 ba\xc3\xb1os completos.- cocina equipada- lavadero - dependencia- living - comedor - recepci\xc3\xb3n - - terraza privada exclusiva. - balc\xc3\xb3n terraza - parrilla.- (cochera privada opcional).- detalles:- paredes de hormig\xc3\xb3n celular t\xc3\xa9rmica y  acustica.- instalaci\xc3\xb3n ca\xc3\xb1er\xc3\xadas de gas epoxi o sigas.- instalaci\xc3\xb3n agua fr\xc3\xada y caliente termo fusi\xc3\xb3n.- yeso alpress.- aberturas dvh alta prestaci\xc3\xb3n herm\xc3\xa9tico.   - puerta blindada.- barandas de vidrio y acero inoxidable.- griferia fv. cierre cer\xc3\xa1mico o similar.- artefacto ferrum o similar.- muebles de cocina laqueado, mesada de silestone.- porcelanatto de alta calidad.- isla con horno - anaffe - mesada de silestone.- instalaciones.  - ascensor electromec\xc3\xa1nico, acero inoxidable, 2 velocidades-- instalaci\xc3\xb3n de panele

In [179]:
#En promedio por ambiente, cual es su superficie cubierta?
ambs = range(1,9)
for amb in ambs:
    promedio = data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == amb]["surface_covered_in_m2"].mean()
    desviacion = data_2017_zo_zs_caba[data_2017_zo_zs_caba["rooms"] == amb]["surface_covered_in_m2"].std()
    print "El promedio para: "+str(amb)+" ambientes es: "+str(promedio)+" y su desviacion estandar es :"+str(desviacion)


El promedio para: 1 ambientes es: 59.6756241142 y su desviacion estandar es :149.158556168
El promedio para: 2 ambientes es: 179.139844895 y su desviacion estandar es :19388.9324886
El promedio para: 3 ambientes es: 83.5486098814 y su desviacion estandar es :217.460733598
El promedio para: 4 ambientes es: 134.175352635 y su desviacion estandar es :1303.02361402
El promedio para: 5 ambientes es: 177.677961847 y su desviacion estandar es :93.5442418768
El promedio para: 6 ambientes es: 239.63322884 y su desviacion estandar es :603.79688206
El promedio para: 7 ambientes es: 233.075367647 y su desviacion estandar es :161.952007931
El promedio para: 8 ambientes es: 195.228571429 y su desviacion estandar es :209.740182091


In [180]:
#Primero me quedo con las que tengan < 1500 metros cuadrados 

In [181]:
#Los que tengan mas de 8 ambientes, los pongo como NaN
def filter_ambientes(row):
    ambientes = row["rooms"]
    if ambientes > 8:
        return np.nan
    elif ambientes == 2.5:
        return 3 #Son muy pocos estos dos 
    elif ambientes == 3.5:
        return 4 
    else:
        return ambientes

In [182]:
data_2017_zo_zs_caba["rooms"] = data_2017_zo_zs_caba.apply(lambda row: filter_ambientes(row), axis = 1)

/home/tomas/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [183]:
data_2017_zo_zs_caba["rooms"].value_counts()

2.0    29224
3.0    23647
1.0    19034
4.0    13933
5.0     4125
6.0     1325
7.0      550
8.0      293
Name: rooms, dtype: int64

In [275]:
print data_2017_zo_zs_caba["surface_total_in_m2"].max()

7832.0


In [276]:

data_2017_zo_zs_caba =data_2017_zo_zs_caba[data_2017_zo_zs_caba["surface_total_in_m2"] != data_2017_zo_zs_caba["surface_total_in_m2"].max()]
data_2017_zo_zs_caba[data_2017_zo_zs_caba["surface_total_in_m2"] == data_2017_zo_zs_caba["surface_total_in_m2"].max()]["description"].values[0]

'c.a.b.a - boca - precio a consultar - c\xc3\x93digo 0005venta con renta. espectacular dep\xc3\xb3sito premium en esquina de 7700 m2 construidos, completamente equipado y con oficinas de primer nivel. sobre terreno de 5500 m2 (120 mts x 45 mts.) techos  altos de 12 mts parab\xc3\xb3lico con aislaci\xc3\xb3n, playa de maniobras, docks de carga sobre nivel, salida a 3 calles. en excelente ubicaci\xc3\xb3n a mts. de autopista 9 de julio sur y av. regimiento de patricios. zonificaci\xc3\xb3n i1 2: distrito industrial, industrial exclusivo. '

In [149]:
#De los que no tenemos datos de la superficie cubierta, volvemos a eliminarlo porque no se pueden recuperar

In [277]:
def filter_surface_total_in_m2(row):
    surface_total = row["surface_total_in_m2"]
    if surface_total > 9500:
        return np.nan
    elif surface_total == 0:
        return np.nan
    else:
        return surface_total

In [278]:
def filter_surface_covered_in_m2(row):
    surface_covered = row["surface_covered_in_m2"]
    if surface_covered > 3000: 
        return np.nan
#https://www.clarin.com/urbano/cada-vez-vive-mas-apretado-tres_dimensiones-miguel_jurado_0_S1OIXFtvXg.html
#un monoambiente tiene como minimo 29 mts2
    elif surface_covered < 29:
        return np.nan
    else:
        return surface_covered


In [279]:
data_2017_zo_zs_caba["surface_total_in_m2"] = data_2017_zo_zs_caba.apply(lambda row: filter_surface_total_in_m2(row), axis = 1)

In [280]:
data_2017_zo_zs_caba["surface_covered_in_m2"] = data_2017_zo_zs_caba.apply(lambda row: filter_surface_covered_in_m2(row), axis = 1)

In [282]:
data_2017_zo_zs_caba[data_2017_zo_zs_caba["surface_total_in_m2"] == data_2017_zo_zs_caba["surface_total_in_m2"].max()]["description"].values[0]

'c.a.b.a - boca - precio a consultar - c\xc3\x93digo 0005venta con renta. espectacular dep\xc3\xb3sito premium en esquina de 7700 m2 construidos, completamente equipado y con oficinas de primer nivel. sobre terreno de 5500 m2 (120 mts x 45 mts.) techos  altos de 12 mts parab\xc3\xb3lico con aislaci\xc3\xb3n, playa de maniobras, docks de carga sobre nivel, salida a 3 calles. en excelente ubicaci\xc3\xb3n a mts. de autopista 9 de julio sur y av. regimiento de patricios. zonificaci\xc3\xb3n i1 2: distrito industrial, industrial exclusivo. '